training loss , val loss curve
plotting the distribution
clean data even more
visualize the tree ? 
tree pruning ? 
show results after every technique

In [ ]:
import numpy as np
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

from sklearn.tree import DecisionTreeClassifier
from keras import models, layers

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, multilabel_confusion_matrix, f1_score
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer


import matplotlib.pyplot as plt

from sklearn.tree import plot_tree
from sklearn.feature_extraction import text
from sklearn.feature_selection import SelectKBest, SelectFromModel
from sklearn.feature_selection import chi2
from keras.backend import clear_session

keywords : extract keywords from a movie plot
eval_model : return score for validation data, test data and predictions
overview : print average precision for val and test, average recall for val and test, average f_score and confusion matrices 
test_score by genres : precision and recall by genres in val and then test

In [ ]:
X = pd.read_csv('/kaggle/input/movie-plot-title-data/data.csv', index_col = 'Unnamed: 0')['0']
y = pd.read_csv('/kaggle/input/movie-plot-title-data/labels.csv', index_col = 'Unnamed: 0')

y = y.iloc[:,np.where(y.sum(axis = 0) > 800)[0]]

X.drop(np.where(y.sum(axis = 1) == 0)[0], inplace = True)
y.drop(np.where(y.sum(axis = 1) == 0)[0], inplace =True)
genre_list =list(y.columns)

In [ ]:
num_words = 1000
tfidf_vec_ = TfidfVectorizer(lowercase = True, ngram_range = (1,1), analyzer = 'word',stop_words = 'english',max_features = num_words)

xtrain, xval_test, ytrain, yval_test = train_test_split(a,b, random_state = 0, train_size = 0.6)
xval, xtest, yval, ytest = train_test_split(xval_test, yval_test, random_state = 0, train_size = 0.5)

xtrain = tfidf_vec_.fit_transform(xtrain).toarray()
xval = tfidf_vec_.transform(xval).toarray()
xtest = tfidf_vec_.transform(xtest).toarray()

In [ ]:
score = []
for depth in [20,40,60,80,100,120,140,160,200,220]:
    tree = DecisionTreeClassifier(criterion = 'gini',splitter = 'best', max_depth = depth ,
                                                        max_features = 'auto', min_samples_split = 2,
                                                        min_samples_leaf = 1, min_impurity_decrease= 0.0, class_weight = 'balanced')

    model = OneVsRestClassifier(tree)
    model.fit(xtrain, ytrain)
    val_preds = model.predict(xval)
    score.append(precision_recall_fscore_support(yval, val_preds, average=None)[2])


In [ ]:
fit = plt.figure(figsize = (10,10))
plt.plot([20,40,60,80,100,120,140,160,200,220], [np.mean(i) for i in score])
plt.xlabel('max_depth')
plt.ylabel('mean validation f1 score')


In [ ]:
score = []
for min_split in [2,5,10,15,20,50,100]:
    tree = DecisionTreeClassifier(criterion = 'gini',splitter = 'best', min_samples_split = min_split , max_depth = 140,
                                                        max_features = 'auto',
                                                        min_samples_leaf = 1, min_impurity_decrease= 0.0, class_weight = 'balanced')


    model = OneVsRestClassifier(tree)
    model.fit(xtrain, ytrain)
    val_preds = model.predict(xval)
    score.append(precision_recall_fscore_support(yval, val_preds, average=None)[2])


In [ ]:
fit = plt.figure(figsize = (10,10))
plt.plot([2,5,10,15,20,50,100], [np.mean(i) for i in score])
plt.xlabel('min_samples_split')
plt.ylabel('mean validation f1 score')


In [ ]:
score = []
for min_leaf in [2,5,10,15,20,40,80]:
    tree = DecisionTreeClassifier(criterion = 'gini',splitter = 'best', min_samples_split = 5 , max_depth = 40,
                                                        max_features = 'auto',
                                                        min_samples_leaf = min_leaf, min_impurity_decrease= 0.0, class_weight = 'balanced')


    model = OneVsRestClassifier(tree)
    model.fit(xtrain, ytrain)
    val_preds = model.predict(xval)
    score.append(precision_recall_fscore_support(yval, val_preds, average=None)[2])


In [ ]:
fit = plt.figure(figsize = (10,10))
plt.plot([2,5,10,15,20,40,80], [np.mean(i) for i in score])
plt.xlabel('min_sample_leaf')
plt.ylabel('mean validation f1 score')


In [ ]:
tree = DecisionTreeClassifier(criterion = 'gini',splitter = 'best', max_depth = 500 ,
                                                        max_features = 1, min_samples_split = 5,
                                                        min_samples_leaf = 1, min_impurity_decrease= 0.0, class_weight = 'balanced')


model = OneVsRestClassifier(tree)
print('fitting')
model.fit(xtrain, ytrain)
print('done')
val_preds = model.predict(xval)
print('done validating')
preds = model.predict(xtest)
print('done predicting')
val_score = precision_recall_fscore_support(yval, val_preds, average=None)
test_score = precision_recall_fscore_support(ytest, preds, average = None)


In [ ]:
print('Train data f1 score:' ,precision_recall_fscore_support(ytrain,model.predict(xtrain), average = None )[2].mean() )

In [ ]:
fig = plt.figure(figsize = (10,10))
plt.bar(y.columns, val_score[2])
